In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.feature import hog, local_binary_pattern
from skimage.filters import gabor_kernel

# Define function to extract LBP features
def extract_lbp_features(image):
    # define the LBP parameters
    radius = 3
    n_points = 8 * radius
    method = 'uniform'
    
    # compute the LBP features for the image
    lbp = local_binary_pattern(image, n_points, radius, method)
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    
    # return the features as a 1D NumPy array
    return hist.ravel()

# Define function to extract HOG features
def extract_hog_features(image):
    # define the HOG parameters
    orientations = 9
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)
    visualize = False
    
    # compute the HOG features for the image
    hog_features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                       cells_per_block=cells_per_block, visualize=visualize)
    
    # return the features as a 1D NumPy array
    return hog_features.ravel()

# Define function to extract Gabor features
def extract_gabor_features(image):
    # define the Gabor parameters
    freqs = [0.05, 0.1, 0.2]
    thetas = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    kernels = []
    
    # create the Gabor kernels
    for freq in freqs:
        for theta in thetas:
            kernel = np.real(gabor_kernel(freq, theta=theta))
            kernels.append(kernel)
            
    # compute the Gabor features for the image
    gabor_features = np.zeros((len(kernels), 2), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        gabor_features[k, 0] = filtered.mean()
        gabor_features[k, 1] = filtered.var()
    
    # return the features as a 1D NumPy array
    return gabor_features.ravel()

# Define the model architecture
input_shape = (width, height, 1)
input_layer = Input(shape=input_shape)

# Convolutional layers for image data
x = Conv2D(32, kernel_size=(3,3), activation='relu')(input_layer)
x = MaxPooling2D()(x)
x = Conv2D(64, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Conv2D(128, kernel_size=(3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)

# Fully connected layers for LBP, HOG, and Gabor features


In [ ]:
# Fully connected layers for LBP, HOG, and Gabor features
lbp_input = Input(shape=(26,), name='lbp_input')
hog_input = Input(shape=(3780,), name='hog_input')
gabor_input = Input(shape=(15,), name='gabor_input')

lbp_dense = Dense(64, activation='relu')(lbp_input)
hog_dense = Dense(64, activation='relu')(hog_input)
gabor_dense = Dense(64, activation='relu')(gabor_input)

# Concatenate the convolutional and fully connected layers
merged = tf.keras.layers.concatenate([x, lbp_dense, hog_dense, gabor_dense])

# Add additional fully connected layers
merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.5)(merged)
merged = Dense(128, activation='relu')(merged)
merged = Dropout(0.5)(merged)
output_layer = Dense(num_classes, activation='softmax')(merged)

# Define the complete model
model = Model(inputs=[input_layer, lbp_input, hog_input, gabor_input], outputs=output_layer)

# Define the optimizer, loss function, and metrics
optimizer = Adam(learning_rate=0.001)
loss_function = 'categorical_crossentropy'
metrics = ['accuracy']

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Define the early stopping callback
early_stopping = EarlyStopping(patience=5)

# Define the data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1,
height_shift_range=0.1, shear_range=0.1, zoom_range=0.1,
horizontal_flip=True, vertical_flip=False, fill_mode='nearest',
preprocessing_function=extract_lbp_features)

val_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=extract_lbp_features)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(width, height),
color_mode='grayscale', batch_size=batch_size,
class_mode='categorical', shuffle=True)

val_generator = val_datagen.flow_from_directory(val_dir, target_size=(width, height),
color_mode='grayscale', batch_size=batch_size,
class_mode='categorical', shuffle=False)

# Train the model
history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator,
callbacks=[early_stopping], verbose=1)